In [1]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re 
from urllib.parse import urljoin
from pymongo import MongoClient
import time


jacky0603.synology.me:28017

In [89]:
db_indict,db_judg=db_connect('jacky0603.synology.me',28017,'indicitment','judgement')
db_indict.remove()
db_judg.remove()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


{'n': 46, 'ok': 1.0}

In [29]:
db_indict,db_judg=db_connect('jacky0603.synology.me',28017,'indicitment','judgement')

a=db_indict.find_one({"字號":"106,偵,22638"})
a['字號']

'106,偵,22638'

In [30]:
#main funcation

#變數初始化
#------------------------------------------------------------------------
#要抓取的頁數
page_num=600
#起始頁
start_page_num=300
base_url='https://lawsnote.com/search/indictment/%E5%82%B7%E5%AE%B3/'
log=False
#------------------------------------------------------------------------
#建立DB連線
db_indict,db_judg=db_connect('jacky0603.synology.me',28017,'indicitment_2','judgement_2')

driver=webdriver.Chrome(r'C:\Users\user\chromedriver')

for i in range(start_page_num,page_num):
    url=urljoin(base_url,str(i))
    driver.get(url)
    try:
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, "search-result__main")))
        time.sleep(3)
        print('第',i,'頁  :')
        #辦斷是否登入
        if log==False:
            driver=login(driver)
            time.sleep(3)
            log=True
        #開始解析
        html=driver.page_source
        soup=bs(html,'lxml')
        #抓出頁面中的判決書連結
        sub_item=catch_href(soup) 
        #篩出府和條件的連結
        indictment=search_fit_indictment(sub_item) 
        #若該頁有符合條件的起訴書-->開始爬起訴書
        if len(indictment)!=0:
            print('indict:',len(indictment),'筆\n')
            parse(driver,indictment,i,db_indict,db_judg)
            
    except Exception as e:
        log_error(i,url,e)
        driver.close()
    
log_record(True)#結束符號
      

第 300 頁  :
indict: 8 筆

起訴書開啟失敗 msg: Message: no such element: Unable to locate element: {"method":"link text","selector":"臺灣臺北地方法院107年度審交易字第579號判決"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.44.609538 (b655c5a60b0b544917107a59d4153d4bf78e1b90),platform=Windows NT 10.0.17134 x86_64)

起訴書開啟失敗 msg: Message: no such element: Unable to locate element: {"method":"link text","selector":"臺灣臺北地方法院107年度審易字第1734號判決"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.44.609538 (b655c5a60b0b544917107a59d4153d4bf78e1b90),platform=Windows NT 10.0.17134 x86_64)



KeyboardInterrupt: 

In [2]:
#解析網頁
def parse(driver,indictment,page_num,db_indict,db_judg):
    fail_open=[]
    judg_fail_open=[]
    parse_base_url='https://lawsnote.com/'
    db_insert_counter=0
    
    for i in range(0,len(indictment)):
        #合成起訴書完整url
        parse_url=urljoin(parse_base_url,indictment[i])
        driver.get(parse_url)
        try:
            WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME,'main-view')))
            time.sleep(3)
            #parse起訴書
            judgement=parse_indictment(driver)
            #進入判決書
            falg,driver=enter_judgement_page(driver)
            #開啟成功->parse判決書
            if falg==True:
                j_judgement=parse_judgement(driver)
            #開啟失敗->紀錄在judg_fail->跳下一筆
            elif falg==False:
                judg_fail_open=judgement['判決書']
                continue
            #起訴書、判決書--存入DB
            judgement.update({'判決法條':j_judgement['相關法條'],'判決法條內文':j_judgement['法條內文']})
            #insert_to_db(judgement,j_judgement)
            insert_to_db(judgement,j_judgement,db_indict,db_judg)
            #print("insert db")
            db_insert_counter+=1
        except Exception as e:
            #起訴書開啟失敗-->記錄在fail_open list
            fail_open.append(indictment[i])
            print('起訴書開啟失敗 msg:',e)
            continue  
    print("insert db:",db_insert_counter)
    
    #判斷是否有錯誤紀錄
    if len(fail_open)!=0 or len(judg_fail_open)!=0:
        log_record(False,fail_open,judg_fail_open,i)
        

In [3]:
def parse_judgement(driver):
    soup=bs(driver.page_source,'lxml')
    #判決書處理title
    j_t_label=soup.select('.document__section.document__meta-content')
    j_t_text=j_t_label[0].find_all('li')
    j_judgement={}
    j_judgement['字號']=j_t_text[0].text.replace('【裁判字號】','')
    j_judgement['日期']=j_t_text[1].text.replace('【裁判日期】','')
    j_judgement['案由']=j_t_text[2].text.replace('【裁判案由】','')
    #判決書內文 judgment_text
    judgment_text=soup.select('.document__section.document__judgement-content')[0].text
    j_judgement['裁判內文']=judgment_text
    #判決書相關法條
    j_related_Issue=soup.select('.document-misc__sub-container.document-misc__relatedIssue-container')
    j=j_related_Issue[0].select('.document-misc__item-container.judgement-misc-content')
    #判決書相關法條標題 j_rel_Issue_title
    j_rel_Issue_title=[x.find('div').text for x in j]
    #判決書相關法條內文  {法條名:內文} j_rel_Issue_content
    j_rel_Issue_content={}
    for i in range(len(j_rel_Issue_title)):
        j_rel_Issue_content[j_rel_Issue_title[i]]=j[i].find_all('ul')[0].text

    #判決書dict key:字號,日期,案由,裁判內文,相關法條標題:list,相關法條內文:{法條名:內文}
    j_judgement['相關法條']=j_rel_Issue_title
    j_judgement['法條內文']=j_rel_Issue_content
    return j_judgement


In [4]:
#錯誤紀錄
def log_record(is_end,fail_open=None,judg_fail_open=None,page_num=None):
    localtime = time.asctime( time.localtime(time.time()) )
    #起訴書開啟失敗紀錄
    with open('fail_open.txt','a') as f:
        if not is_end:
            for i in fail_open:
                f.write("{} 第{}頁  href:{}\n".format(localtime,page_num,i))
        else:
            f.write('-------------------------------------------\n')
    #判決書開啟失敗紀錄         
    with open('judg_fail_open.txt','a') as f:
        if not is_end:
            for i in judg_fail_open:
                f.write("{} 第{}頁  開啟失敗判決書:{}\n".format(localtime,page_num,i))
        else:
            f.write('--------------------------------------------\n')
    return

def log_error(page_num,error_url,exp_msg):
    localtime = time.asctime( time.localtime(time.time()) )
    with open('crawler_error.txt','a') as f:
        f.write("{} 第{}頁 href:{} 錯誤msg:{}\n",format(localtime,page_num,error_url,exp_msg))

In [5]:
#DB
def db_connect(mongo_url,port,indic_name,judg_name):
    try:
        client=MongoClient(mongo_url,port)
        db=client['law']
        indicit=db[indic_name]
        judg=db[judg_name] 
    except Exception as e:
        print('db_connect',e)
    return indicit,judg
                    
def insert_to_db(judgement,j_judgement,db_indict,db_judg):
    try:
        db_indict.insert_one(judgement)
        db_judg.insert_one(j_judgement)
    except Exception as e:
        print('db insert fail:',e)
        

In [6]:
def enter_judgement_page(driver):
    soup=bs(driver.page_source,'lxml')
    judg=soup.select('.document-misc__sub-container.document-misc__judgementHistory-container')
    judg[0].find('a')['id']
    driver.find_element_by_link_text(judg[0].find('a')['id']).click()
    try:
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME,'main-view')))
    except:
        return False,driver
    return True,driver


In [7]:
def parse_indictment(driver):
    soup=bs(driver.page_source,'lxml')
    #起訴書處理title
    title_label=soup.select('.document__section.document__meta-content')
    title_text=title_label[0].find_all('li')
    judgement={}
    judgement['字號']=title_text[0].text.replace('【字號】','')
    judgement['日期']=title_text[1].text.replace('【日期】','')
    judgement['案由']=title_text[2].text.replace('【案由】','')
    #起訴書處理內文 content_label
    content_label=soup.select('.document__section.document__judgement-content')
    judgement['起訴書內文']=content_label[0].text
    #起訴書相關法條
    related_Issue=soup.select('.document-misc__sub-container.document-misc__relatedIssue-container')
    r=related_Issue[0].select('.document-misc__item-container.indictment-misc-content')
    #起訴書相關法條標題 related_Issue_title
    related_Issue_title=[x.find('div').text for x in r]
    #起訴書相關法條內文  {法條名:內文} related_Issue_content
    related_Issue_content={}
    for i in range(len(related_Issue_title)):
        related_Issue_content[related_Issue_title[i]]=r[i].find_all('ul')[0].text

    #起訴書dict key:字號,日期,案由,起訴書內文,相關法條標題:list,相關法條內文:{法條名:內文},判決書
    judgement['相關法條']=related_Issue_title
    judgement['法條內文']=related_Issue_content
    judg=soup.select('.document-misc__sub-container.document-misc__judgementHistory-container')
    judgement['判決書']=judg[0].find('a')['id']
    return judgement

In [8]:
def catch_href(soup):
    sub_item=soup.find_all('a',{'class':'search-result__item-link'})
    return sub_item

def search_fit_indictment(sub_item):
    indictment_item=[]
    for s in sub_item:
        if s.text.find('傷害')!=-1 and s.text.find('起訴書')!=-1:
            indictment_item.append(s['href'])
    return indictment_item

def login(driver):
    driver.find_element_by_xpath(u"(.//*[normalize-space(text()) and normalize-space(.)='揪團'])[1]/preceding::button[1]").click()
    #driver.find_element_by_xpath(u"//*[@id=\"root\"]/div/div/header/div[4]/div/button[1]").click()
    driver.find_element_by_id("email").click()
    driver.find_element_by_id("email").clear()
    driver.find_element_by_id("email").send_keys("s1022756@gm.pu.edu.tw")
    time.sleep(1)
    driver.find_element_by_id("password").clear()
    driver.find_element_by_id("password").send_keys("asd753")
    time.sleep(1)
    driver.find_element_by_xpath(u"(.//*[normalize-space(text()) and normalize-space(.)='忘記密碼？'])[1]/following::button[1]").click()
    return driver